<a href="https://colab.research.google.com/github/BarelHeby/Deep-Learning---Chest-X-Ray/blob/main/Deep_Learning_Chest_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---
**FIRST SETUP**

In [1]:
!pip install -q kaggle
from google.colab import files

# Create a Kaggle API token and save it to a file.
!echo '{"username":"barelheby","key":"978bdd6cd1cc991c69bbe920fe75a9cc"}' > kaggle.json

# Copy the Kaggle API token file to the correct location.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json

# Change the permissions of the Kaggle API token file.
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle.
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

# Unzip the downloaded dataset.
!unzip chest-xray-pneumonia.zip

# Print the contents of the current working directory.
!ls


Streaming output truncated to the last 5000 lines.
  inflating: chest_xray/train/NORMAL/IM-0435-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0437-0001-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0437-0001-0002.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0437-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0438-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0439-0001-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0439-0001-0002.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0439-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0440-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0441-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0442-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0444-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0445-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0446-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0447-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0448-0001.jpeg  
  inflating: ches

---
**IMPORTS**

In [227]:
import pandas as pd
from os.path import join
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet import ResNet152
from keras.layers import BatchNormalization,Dropout,Dense,Flatten
from tensorflow.keras.optimizers import Adamax,Adam
import os
import shutil
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import plotly.express as px
from sklearn.manifold import TSNE




---
 **Configuration**


In [3]:
TF_BATCH_SIZE = 32
TF_SEED = 123
IMG_HEIGHT = 256
IMG_WIDTH = 256
DIR_DATASET = "/content/chest_xray"

---
**Normalize Image Function**



---

**Training Set Preprocess**

In [4]:
train_directory = join(DIR_DATASET,"train")
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    validation_split=0.2,
    subset = "training",
    seed = TF_SEED,
    image_size = (IMG_HEIGHT,IMG_WIDTH),
    batch_size = TF_BATCH_SIZE,
)

Found 5216 files belonging to 2 classes.
Using 4173 files for training.


---

**Validation Set Prep**

In [9]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    validation_split=0.2,
    subset = "validation",
    seed = TF_SEED,
    image_size = (IMG_HEIGHT,IMG_WIDTH),
    batch_size = TF_BATCH_SIZE
)

Found 5216 files belonging to 2 classes.
Using 1043 files for validation.


In [5]:
train_dataset.class_names

['NORMAL', 'PNEUMONIA']

In [10]:
train_dataset = train_dataset.map(lambda x,y:(x/255.0,y))
validation_dataset = validation_dataset.map(lambda x,y:(x/255.0,y))

---
**Part 1 Model**

In [11]:
base_network = VGG16(weights="imagenet",include_top=False,input_shape = (IMG_HEIGHT,IMG_WIDTH,3),pooling="avg")
base_network.trainable = False

58889256/58889256 [==============================] - 2s 0us/step


In [12]:
model_assigment_1  = Sequential(name="Assigment_1")
model_assigment_1.add(base_network)
model_assigment_1.add(Flatten())
model_assigment_1.add(Dense(512,activation="relu"))
model_assigment_1.add(Dropout(0.50))
model_assigment_1.add(Dense(128,activation="relu"))
model_assigment_1.add(Dropout(0.25))
model_assigment_1.add(Dense(60,activation="relu"))
model_assigment_1.add(Dense(1,activation="sigmoid"))
model_assigment_1.compile(optimizer=Adam(),loss='binary_crossentropy', metrics=['accuracy'])
model_assigment_1.summary()

Model: "Assigment_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 60)                

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
history = model_assigment_1.fit(train_dataset,
                        epochs= 20,
                        validation_data = validation_dataset,
                        callbacks = early_stopping)

Epoch 1/20
131/131 [==============================] - 40s 290ms/step - loss: 0.1142 - accuracy: 0.9573 - val_loss: 0.0949 - val_accuracy: 0.9645
Epoch 2/20
131/131 [==============================] - 39s 288ms/step - loss: 0.1141 - accuracy: 0.9576 - val_loss: 0.0704 - val_accuracy: 0.9760
Epoch 3/20
131/131 [==============================] - 41s 298ms/step - loss: 0.1167 - accuracy: 0.9547 - val_loss: 0.0878 - val_accuracy: 0.9712
Epoch 4/20
131/131 [==============================] - 38s 280ms/step - loss: 0.1156 - accuracy: 0.9557 - val_loss: 0.0575 - val_accuracy: 0.9770
Epoch 5/20
131/131 [==============================] - 42s 301ms/step - loss: 0.1054 - accuracy: 0.9600 - val_loss: 0.0602 - val_accuracy: 0.9789
Epoch 6/20
131/131 [==============================] - 41s 298ms/step - loss: 0.1116 - accuracy: 0.9607 - val_loss: 0.0564 - val_accuracy: 0.9770
Epoch 7/20
131/131 [==============================] - 41s 300ms/step - loss: 0.1087 - accuracy: 0.9609 - val_loss: 0.0588 - val_ac

In [ ]:
validation_loss, validation_accuracy = model_assigment_1.evaluate(validation_dataset)
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)

33/33 [==============================] - 8s 210ms/step - loss: 0.0564 - accuracy: 0.9808
Validation Loss: 0.05635236203670502
Validation Accuracy: 0.9808245301246643


In [ ]:
model_assigment_1.save("model_assigment_1.keras")

In [13]:
DIR_ASS_2 = join(DIR_DATASET,"assiment_2_dataset")
DIR_ASS_2_BACTERIA  = join(DIR_ASS_2,"BACTERIA")
DIR_ASS_2_VIRUS = join(DIR_ASS_2,"VIRUS")
DIR_ASS_2_NORMAL = join(DIR_ASS_2,"NORMAL")


In [14]:
try:
  shutil.rmtree(DIR_ASS_2)
except Exception as e:
  print(e)
try:
  os.mkdir(DIR_ASS_2)
  os.mkdir(DIR_ASS_2_BACTERIA)
  os.mkdir(DIR_ASS_2_VIRUS)
  os.mkdir(DIR_ASS_2_NORMAL)
except Exception as e:
  print(e)

[Errno 2] No such file or directory: '/content/chest_xray/assiment_2_dataset'


---
**Copy Images From Source To New Folder**

Copy sick

In [15]:
# prompt: copy all files from DIR_DATASET/test/PNEUMONIA to DIR_DATASET/assigment_2_dataset
source_dir = join(DIR_DATASET,"train","PNEUMONIA")
for filename in os.listdir(source_dir):
  if "bacteria" in filename.lower():
    dest_dir  = DIR_ASS_2_BACTERIA
  elif "virus" in filename.lower():
    dest_dir = DIR_ASS_2_VIRUS
  else:
    pass
  shutil.copy(join(source_dir,filename),dest_dir)



Copy normal

In [16]:
source_dir = join(DIR_DATASET,"train","NORMAL")
for filename in os.listdir(source_dir):
  shutil.copy(join(source_dir,filename),DIR_ASS_2_NORMAL)

---
**Assigment 2 preprocess**

In [17]:
train_dataset_ass_2 = tf.keras.utils.image_dataset_from_directory(
    DIR_ASS_2,
    validation_split=0.2,
    subset = "training",
    seed = TF_SEED,
    image_size = (IMG_HEIGHT,IMG_WIDTH),
    batch_size = TF_BATCH_SIZE,
    label_mode = "categorical",
)
validation_dataset_ass_2 = tf.keras.utils.image_dataset_from_directory(
    DIR_ASS_2,
    validation_split=0.2,
    subset = "validation",
    seed = TF_SEED,
    image_size = (IMG_HEIGHT,IMG_WIDTH),
    batch_size = TF_BATCH_SIZE,
    label_mode = "categorical",
)

Found 5216 files belonging to 3 classes.
Using 4173 files for training.
Found 5216 files belonging to 3 classes.
Using 1043 files for validation.


In [18]:
train_dataset_ass_2.class_names

['BACTERIA', 'NORMAL', 'VIRUS']

In [ ]:
# train_dataset_ass_2 = train_dataset_ass_2.map(lambda x,y:(x/255.0,y))
# validation_dataset_ass_2 = validation_dataset_ass_2.map(lambda x,y:(x/255.0,y))

In [19]:
base_network_ass_2 = ResNet152(include_top=False,input_shape = (IMG_HEIGHT,IMG_WIDTH,3),pooling="avg")
base_network_ass_2.trainable = False

234698864/234698864 [==============================] - 6s 0us/step


In [21]:
model_assigment_2  = Sequential(name="Assigment_2")
model_assigment_2.add(base_network_ass_2)
# model_assigment_2.add(BatchNormalization())
# model_assigment_2.add(Flatten())
# model_assigment_2.add(Dense(512,activation="relu"))
model_assigment_2.add(Dense(1024,activation="relu"))
model_assigment_2.add(Dropout(0.3))
# model_assigment_2.add(BatchNormalization())
model_assigment_2.add(Dense(512,activation="relu"))
# model_assigment_2.add(Dense(256,activation="relu"))
model_assigment_2.add(Dropout(0.2))
# model_assigment_2.add(Dense(128,activation="relu"))
model_assigment_2.add(Dense(64,activation="relu"))
model_assigment_2.add(Dense(3,activation="softmax"))
model_assigment_2.compile(optimizer=Adam(),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_assigment_2.summary()

Model: "Assigment_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 2048)              58370944  
                                                                 
 dense_4 (Dense)             (None, 1024)              2098176   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 3)                 

In [22]:
early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=5, restore_best_weights=True)
history_ass_2 = model_assigment_2.fit(train_dataset_ass_2,
                        epochs= 50,
                        validation_data = validation_dataset_ass_2,
                        callbacks = early_stopping)

Epoch 1/50
131/131 [==============================] - 69s 406ms/step - loss: 0.6818 - categorical_accuracy: 0.7100 - val_loss: 0.5375 - val_categorical_accuracy: 0.7852
Epoch 2/50
131/131 [==============================] - 48s 355ms/step - loss: 0.5011 - categorical_accuracy: 0.7815 - val_loss: 0.4832 - val_categorical_accuracy: 0.7948
Epoch 3/50
131/131 [==============================] - 48s 352ms/step - loss: 0.4987 - categorical_accuracy: 0.7886 - val_loss: 0.5016 - val_categorical_accuracy: 0.7891
Epoch 4/50
131/131 [==============================] - 47s 348ms/step - loss: 0.4582 - categorical_accuracy: 0.8059 - val_loss: 0.5025 - val_categorical_accuracy: 0.7939
Epoch 5/50
131/131 [==============================] - 49s 359ms/step - loss: 0.4516 - categorical_accuracy: 0.8081 - val_loss: 0.4465 - val_categorical_accuracy: 0.8121
Epoch 6/50
131/131 [==============================] - 47s 348ms/step - loss: 0.4330 - categorical_accuracy: 0.8140 - val_loss: 0.4753 - val_categorical_acc

In [23]:
validation_loss, validation_accuracy = model_assigment_2.evaluate(validation_dataset_ass_2)
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)

33/33 [==============================] - 10s 265ms/step - loss: 0.4465 - categorical_accuracy: 0.8121
Validation Loss: 0.44653642177581787
Validation Accuracy: 0.8120805621147156


In [25]:
model_assigment_2.save("model_assigment_2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**Assigment 3**

---



Train The KNN Classifier

In [119]:
X_train = []
y_train = []
for batch, label in train_dataset_ass_2:
    X_train.append(model_assigment_2.predict(batch))
    y_train.append(label)
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)




KNeighborsClassifier()

Function To Classify New Image

In [121]:
def classify(new_image_path,model,knn_model):
  new_image = Image.open(new_image_path)
  new_image = new_image.resize((IMG_WIDTH,IMG_HEIGHT))
  new_image_array = np.array(new_image)
  new_image_array = np.expand_dims(new_image_array,axis=0)
  new_image_array = np.expand_dims(new_image_array,axis=-1)
  new_image_array = np.repeat(new_image_array, 3, axis=-1)
  new_image_tensor = tf.convert_to_tensor(new_image_array,tf.float32)
  # Create the embedding vector for the image
  new_image_embedding = model.predict(new_image_tensor)
  # Use knn to predict
  y_pred_knn = knn_model.predict(new_image_embedding)[0]
  for i in range(len(y_pred_knn)):
    if int(y_pred_knn[i])==1:
      return train_dataset_ass_2.class_names[i]

Predict New Image Using KNN  

In [128]:
img_1 = "/content/chest_xray/test/PNEUMONIA/person173_bacteria_830.jpeg"
print("Expected: BACTERIA, Got:", classify(img_1,model_assigment_2,knn))
img_2 = "/content/chest_xray/test/PNEUMONIA/person17_virus_48.jpeg"
print("Expected: VIRUS, Got:", classify(img_2,model_assigment_2,knn))
img_3 = "/content/chest_xray/test/NORMAL/IM-0011-0001.jpeg"
print("Expected: NORMAL, Got:", classify(img_3,model_assigment_2,knn))

1/1 [==============================] - 0s 32ms/step
Expected: BACTERIA, Got: BACTERIA
1/1 [==============================] - 0s 30ms/step
Expected: VIRUS, Got: VIRUS
1/1 [==============================] - 0s 76ms/step
Expected: NORMAL, Got: NORMAL


In [228]:
embeddings = model_assigment_2.predict(validation_dataset_ass_2)
tsne_data = TSNE(n_components=3,perplexity=3).fit_transform(embeddings)
df = pd.DataFrame(tsne_data, columns=["x", "y", "z"])
X = df["x"]
y = df["y"]
z = df["z"]



33/33 [==============================] - 11s 297ms/step


In [229]:
y_names = [validation_dataset_ass_2.class_names[np.argmax(row)] for row in tsne_data]
fig = px.scatter_3d(x=X, y=y, z=z, color=y_names, opacity=0.8)
fig.show()
